# Neo4j

## Conexión con Cassandra y borrado de datos

In [ ]:
%load_ext cypher

In [ ]:
from py2neo import Graph, Node, Relationship

graph = Graph()

In [ ]:
# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en Pandas

A partir del fichero Excel desnormalizamos los datos 

In [ ]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

## Carga de datos en Neo4j

La carga se realizará en Neo a partir de la lista de datos en formato JSON, generados con Pandas

In [ ]:
import json

json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

En Neo se van a crear 3 tipos de nodos y 2 tipos de relaciones

<br><br> 

<img src="images/Modelo%20Neo4j.png",width=400,height=300>

<br><br> 

Adicionalmente se van a guardar los siguientes atributos:

**Nodo Comercio**
- Nombre del comercio
- Actividad a la que se dedica
- Categoría de actividad (Podríamos haber creado un tipo de nodo independente)

**Nodo Persona**
- Nombre de la persona

**Nodo Organización**
- Nombre de la organización

Respecto a las relaciones, vamos a almacenar los siguientes atributos:

**Relación COMPRA**
- Fecha en formato UNIX Timestamp (el mismo formato que está en el JSON)
- hora y minuto de la compra
- Importe de la compra

**Relación PERTECENE**
- Función que hace la persona en una organización

Creamos los nodos y relaciones en Neo4j simplemente iterando sobre la lista de JSONs

In [ ]:
import datetime

for movimiento_json in json_list:
    # Nodos
    persona = Node("Persona", nombre=movimiento_json["nombre"])
    comercio = Node("Comercio", nombre=movimiento_json["comercio"], 
                                actividad = movimiento_json["actividad"],
                                actividad_completa = movimiento_json["actividad_completa"])
    organizacion = Node("Organizacion", nombre = movimiento_json["organizacion"])
        
    # Relaciones  
    rel_persona_organizacion = Relationship(persona, "PERTENECE", organizacion,
                                           funcion = movimiento_json["funcion"])

    rel_persona_comercio = Relationship(persona, "COMPRA", comercio, 
            fecha = movimiento_json['fecha'],
            hora =  movimiento_json["hora"],
            minuto =  movimiento_json["minuto"],
            importe = movimiento_json["importe"])
       
    graph.merge(persona | comercio | organizacion | rel_persona_comercio | rel_persona_organizacion)


<br><br> 

<img src="images/Resultado.png",width=800,height=600>

<br><br> 

## Funciones de utilidad para la consulta de datos

Neo4j, en su versión actual, no puede guardar datos de tipo fecha en los atributos, por lo que se almacena en formato TIMESTAMP de Unix (el formato original).  

La siguiente función de utilidad convierte una columna de una DataFrame en formato TIMESTAMP a Fecha

In [ ]:
def transform_date(df, column_name):
    if column_name in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df[column_name] = pd.to_datetime(df[column_name], unit = 'ms')
        
    return df

Función de utilidad para convertir los resultados devueltos por la base de datos en un DataFrame de Pandas

In [ ]:
def get_dataframe(data):
    df = pd.DataFrame(list(data), columns = data.keys())
        
    return df

## Consulta a Neo4j

### Los 10 movimientos mas caros

Para obtener las lista de personas que mas han gastado simplemente relacionamos los 3 tipos de nodos y devolvemos los campos que nos interesan

In [ ]:
%%cypher
MATCH (persona:Persona)-[compra:COMPRA]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, compra.importe
ORDER BY compra.importe DESC
LIMIT 10

Utilizamos la función ** graph.run()** para obtener un ResultSet de datos, que es transformada en un Dataframe de Pandas

In [ ]:
query = """
MATCH (persona:Persona)-[compra:COMPRA]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, compra.importe, compra.fecha
ORDER BY compra.importe DESC
LIMIT 10
""" 

data = graph.run(query)
df = get_dataframe(data)
df = transform_date(df, 'compra.fecha')
df

La misma query que el caso anterior pero utilizando un método distinto.  
El fin es el mismo: Una DataFrame de Pandas

In [ ]:
df = %cypher MATCH (persona:Persona)-[compra:COMPRA]-(comercio:Comercio) \
             RETURN persona.nombre, comercio.actividad_completa, compra.importe \
             ORDER BY compra.importe DESC \
             LIMIT 10
            
df.get_dataframe()

### Importes de una persona agrupados por actividad

En este caso se realiza una agrupación de los datos. Observa que Neo4j lo realiza automáticamente al utilizar una función de agrupación de datos como **SUM()**

In [ ]:
%%cypher
MATCH (persona:Persona)-[compra:COMPRA]-(comercio:Comercio)
WHERE comercio.actividad = 'HOGAR'
RETURN persona.nombre, comercio.actividad_completa, SUM(compra.importe) as importe
ORDER BY importe DESC
LIMIT 10


In [ ]:
query = """
MATCH (persona:Persona)-[compra:COMPRA]-(comercio:Comercio)
WHERE persona.nombre = {name}
RETURN persona.nombre, comercio.actividad, SUM(compra.importe) as importe
ORDER BY importe DESC
"""

data = graph.run(query, name='Mariano Pérez Claver')
get_dataframe(data)